In [1]:
import numpy as np
import math
import random

# Hilfsfunktionen:

In [2]:
def sqm(x, y, m):
    if y == 0:
        return 1
    if y % 2 != 0:
        a = sqm(x, y-1, m)
        return (a * x) % m
    if y % 2 == 0:
        a = sqm(x, y/2, m)
        return (a * a) % m

In [3]:
def eea(a, b):
    if b == 0:
        return (a, 1, 0)
    else:
        (ggt, other_a, other_b) = eea(b, a % b)
        return (ggt, other_b, other_a - (other_b * int((a / b))))

In [4]:
def get_inverse(number, mod):
    for i in range(mod):
        current_value = (number * i) % mod
        if current_value == 1:
            return i
    # -1 als Error wert
    return -1

In [5]:
def is_prime(x):
    if x == 0 or x == 1: return False
    if x == 2: return True
    sqrt_x = math.floor(math.sqrt(x))
    for i in range(3, sqrt_x + 1, 2):
        if x % i == 0:
            return False
    return True

# 1.)
Es seien die Primzahlen $p := 43$ und $q := 67$ gegeben.

## a ) Berechnen eines RSA-Schlüsselpaares

In [6]:
p = 43
q = 67
n = p * q
n

2881

In [7]:
phi_n = (p-1)*(q-1)
phi_n

2772

In [8]:
a = 5 # ggt(5, phi_n) = 1
b = get_inverse(a, phi_n)
b

1109

In [9]:
pub_key = (n, a)
priv_key = (n, b)
pub_key, priv_key

((2881, 5), (2881, 1109))

## b) Entschlüsseln von $m = 2000$ mit dem Schlüsselpaar aus a)

In [10]:
m = 2000
c = sqm(m, a, n)
c

1974

Kontrolle:

In [11]:
checked_m = sqm(c, b, n)
checked_m == m

True

# 2.)
Es seien folgende Zahlen gegeben:

\begin{align*}
a_1 &= 100 \quad &m_1 &= 33 \\
a_2 &= 16 \quad &m_2 &= 91 \\
a_3 &= 101 \quad &m_3 &= 41 \\
a_4 &= 42 \quad &m_4 &= 38 \\
\end{align*}

Und zusätzlich $n = 4678674$.

## a) Primfaktorzerlegung aller $m_i$ und $n$.

In [12]:
a = np.array([100, 16, 101, 42])
m = np.array([33, 91, 41, 38])
n = 4678674

In [13]:
def get_divisors(num):
    divisors = [1, num]
    sqrt_num = math.floor(math.sqrt(num))
    for i in range(2, sqrt_num + 1):
        if num % i == 0:
            divisors.append(i)

    remaining_divisors = [int(num /divisor) for divisor in divisors if int(num / divisor) not in divisors]
    divisors.extend(remaining_divisors)
    divisors = sorted(divisors)
    return divisors

In [14]:
for value in m:
    divisors = get_divisors(value)
    print("Divisors for {}: {}".format(value, divisors))
    divisors.remove(1)
    divisors.remove(value)
    prime_factorization = "*".join(list(map(str, divisors)))
    if not prime_factorization: prime_factorization = str(value)
    print("Prime factorization = " + prime_factorization + "\n")

# Die Primfaktor zerlegung ergibt sich als das Produkt all der oberen Primfaktorzerlegungen, 
# denn alle mi sind zu allen mj, i != j teilerfremd.

Divisors for 33: [1, 3, 11, 33]
Prime factorization = 3*11

Divisors for 91: [1, 7, 13, 91]
Prime factorization = 7*13

Divisors for 41: [1, 41]
Prime factorization = 41

Divisors for 38: [1, 2, 19, 38]
Prime factorization = 2*19



## b) Bestimmen einer Zahl $ x \in \{0, ..., n-1\}$, sodass $x \equiv a_i \mod m_i$ für $1 \leq i \leq 4$ gilt. 

In [15]:
div_m = np.array([int(n / m[i]) for i in range(m.size)])
div_m

array([141778,  51414, 114114, 123123])

In [16]:
r = np.array([eea(div_m[i], m[i])[1] % m[i] for i in range(div_m.size)])
r

array([10, 90, 15, 13])

In [17]:
def chinese_div(r, div_m, a, n):
    sum = 0
    for i in range(r.size):
        sum += a[i] * div_m[i] * r[i]
    return sum % n

In [18]:
x = chinese_div(r, div_m, a, n)
x

2090650

In [19]:
def check_x(a, m, x):
    for i in range(len(a)):
        lhs = a[i] % m[i]
        rhs = x % m[i]
        if lhs != rhs:
            return False
    return True
        

In [20]:
check_x(a, m, x)

True

## c) Angeben eines folgenden Gruppenhomomorphismus

Gruppenismorphismus mit
$$
\mathbb{Z}_n^* \rightarrow (\mathbb{Z}_{m_1}^* \times \mathbb{Z}_{m_2}^* \times \mathbb{Z}_{m_3}^* \times \mathbb{Z}_{m_4}^*)
$$

$$
\Rightarrow g_1(x) := \left(x \mod m_1, x \mod m_2, x \mod m_3, x \mod m_4\right) \\
$$

Beweis:
$$
g_1(x, y) = ((x \cdot y) \mod m_1, ... ) = (x \mod m_1, ...) (y \mod m_1, ...) = g_1(x) \cdot g_1(y)
$$

Isomorph, denn der hier verwendete chinesische Restsatz liefert *genau* eine Zahl $ x $. Damit ist eine Bijektion gegeben und folglich ein Isomorphismus.

## d) Berechnen von $125^{10} \mod n$ unter Verwendung von 8-Bit Zahlen, mit Ausnahme von internen Zahlen des chinesischen Restsatzes.

In [21]:
g1 = np.array([125 % m[i] for i  in range(m.size)])
g1

array([26, 34,  2, 11])

In [22]:
g1_sqm = np.array([sqm(g1[i], 10, m[i]) for i  in range(g1.size)])
g1_sqm

array([ 1, 64, 40, 11])

In [23]:
c_solution = chinese_div(r, div_m, g1_sqm, n)
c_solution

4664815

In [24]:
125**10 % n == c_solution

True

In [25]:
check_x(g1_sqm, m, c_solution)

True

## e) Angeben eines folgenden Gruppenisomorphismus

$n = \prod_{i=1}^k p_i$ sei die Primfaktorzerlegung von $n$. Gesucht ist ein folgender Isomorphismus.
$$
\mathbb{Z}_n^* \rightarrow (\mathbb{Z}_{p_1}^* \times ... \times \mathbb{Z}_{p_k}^*)
$$
Wir setzen also statt $m_i, 1 \leq i \leq 4$ alle $p_i$ ein, die die Primakfakorzerlegung von $n$ darstellen.

$$
\Rightarrow g_2(x) := \left(x \mod p_1, ... , x  \mod p_2\right)
$$

Beweis: 
$$
g_2(x, y) = ((x \cdot y) \mod p_1 ...) = (x \mod p_1, ...)(y \mod p_1, ...) = g_2(x) \cdot g_2(y)
$$

Wir argumentieren hier also analog zum Ismorphismus $g_1$, auch bezüglich der Eigeschaft der Bijektivität.

## Berechnen von $125^{10} \mod n$ mithilfe von $g_2$

In [26]:
p = [get_divisors(value) for value in m]
p = np.array([div for div_list in p for div in div_list if is_prime(div)])
p

array([ 3, 11,  7, 13, 41,  2, 19])

In [27]:
div_p = np.array([int(n / p[i]) for i in range(p.size)])
div_p

array([1559558,  425334,  668382,  359898,  114114, 2339337,  246246])

In [28]:
r_p = np.array([eea(div_p[i], p[i])[1] % p[i] for i in range(div_p.size)])
r_p

array([ 2,  7,  1, 11, 15,  1, 16])

In [29]:
g2 = np.array([125 % p[i] for i  in range(p.size)])
g2

array([ 2,  4,  6,  8,  2,  1, 11])

In [30]:
g2_sqm = np.array([sqm(g2[i], 10, p[i]) for i  in range(g2.size)])
g2_sqm

array([ 1,  1,  1, 12, 40,  1, 11])

In [31]:
g2_solution = chinese_div(r_p, div_p, g2_sqm, n)
g2_solution

4664815

In [32]:
check_x(g2_sqm, p, c_solution)

True

## Bestimmen von Zahlen $q_1, ..., q_k$, sodass $q_i \equiv x \mod p_i$ für alle $i \in \{1, ..., k\}$ und dem $x$ aus b)

Da das $x$ gerade dieses Kongruenzensystem per Definition löst, können wir einfach die Moduli berechnen und die Primärepräsentanten dieser Ergebnisse als $q_i$ angeben.

In [33]:
q = [x % p[i] for i in range(p.size)]
q

[1, 1, 2, 3, 19, 0, 4]

In [34]:
check_x(q, p, x)

True

# 3.)

Es seien die *öffentlichen* RSA-Schlüssel $k_1 = (391, 3)$, $k_2 = (55, 3)$ und $k_3 = (1189, 3)$, sowie die *Chiffretexte* 
$c_1 = 133$, $c_2 = 48$, $c_3 = 659$ gegeben. 
Falls möglich, Hastads Angriff verwenden um den Klartext $m$ zu berechnen. Falls es nicht möglich ist, begründen.

Wir haben 3 Chiffretext mit 3 öffentlichen Schlüsseln gegeben, insofern spricht nichts gegen den Hastad-Angriff.
$a$ ist sogar gegeben als 3, der in allen Nachrichten den öffentlichen Exponenten darstellt.

In [35]:
a = 3
k = np.array([(391, 3), (55, 3), (1189, 3)], dtype="int64")
c = np.array([133, 48, 659], dtype="int64")
a, k, c

(3,
 array([[ 391,    3],
        [  55,    3],
        [1189,    3]], dtype=int64),
 array([133,  48, 659], dtype=int64))

In [36]:
N = np.array([key[0] for key in k], dtype="int64")
N

array([ 391,   55, 1189], dtype=int64)

In [37]:
# Check ggt(ni, nj) == 1, i != j
def check_ggt_for_vector(n):
    for i in range(len(n)):
        for j in range(len(n)):
            if i != j:
                coprime = eea(n[i], n[j])[0] == 1
                if not coprime: return False
    return True

In [38]:
check_ggt_for_vector(N)

True

In [39]:
N_prod = np.prod(N)
N_prod

25569445

Wir können nun den chinesischen Restsatz anwenden, mit entsprechender Vorbereitung:

In [40]:
div_N = np.array([int(N_prod / N[i]) for i in range(N.size)], dtype="int64")
div_N

array([ 65395, 464899,  21505], dtype=int64)

In [41]:
r_N = np.array([eea(div_N[i], N[i])[1] % N[i] for i in range(div_N.size)], dtype="int64")
r_N

array([  4,  24, 658], dtype=int64)

In [42]:
hastat_solution = chinese_div(r_N, div_N, c, N_prod)
hastat_solution

19683

In [43]:
check_x(c, N, hastat_solution)

True

In [44]:
m = int(np.round(np.power(hastat_solution, (1/a))))
m

27

Wir kontrollieren das erhaltene $m = 27$:

In [45]:
gen_c = np.array([sqm(m, a, N[i]) for i in range(N.size)])
np.array_equal(gen_c, c) 

True